# Implement models with PyTorch to allow for GPU cuda compatability

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

In [2]:
def train_model(input_name:str, output_name:str) -> None:
    # load data
    df = pd.read_csv(input_name)
    X = df.drop('Outcome', axis=1)
    y = df['Outcome']

    # convert data to numpy
    X = np.array(X)
    y = np.array(y)

    # split and scale the data
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # convert data to cuda tensors
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32).cuda()
    y_train_tensor = torch.tensor(y_train, dtype=torch.float32).cuda()
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32).cuda()
    y_test_tensor = torch.tensor(y_test, dtype=torch.float32).cuda()

    # create batches for the data to reduce noise
    train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

    # define model (should probably use command line arguments in the future)
    class ANN_Model(nn.Module):
        def __init__(self, input_size):
            super(ANN_Model, self).__init__()
            self.fc1 = nn.Linear(input_size, 256)
            self.fc2 = nn.Linear(256, 128)
            self.fc3 = nn.Linear(128, 64)
            self.fc4 = nn.Linear(64, 32)
            self.fc5 = nn.Linear(32, 1)
            self.relu = nn.ReLU()
            self.dropout = nn.Dropout(0.1)
            
        def forward(self, x):
            x = self.relu(self.fc1(x))
            x = self.dropout(x)
            x = self.relu(self.fc2(x))
            x = self.dropout(x)
            x = self.relu(self.fc3(x))
            x = self.dropout(x)
            x = self.relu(self.fc4(x))
            x = self.dropout(x)
            x = self.fc5(x)
            return x

    # create an instance of the model
    model = ANN_Model(input_size=X_train.shape[1]).cuda()
    device = torch.device("cuda") # replace with "cpu" if this gives error
    model.to(device)

    # define the loss function and optimizer
    loss_function = nn.BCEWithLogitsLoss()
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    # tain the model
    num_epochs = 25
    for epoch in range(num_epochs):
        model.train()
        for batch_inputs, batch_targets in train_loader:
            # move batch to gpu
            batch_inputs = batch_inputs.to(device)
            batch_targets = batch_targets.to(device)
            
            # forward pass
            outputs = model(batch_inputs)
            loss = loss_function(outputs, batch_targets.unsqueeze(1))
            
            # backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
        # print status
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}")

    # print accuracy
    model.eval()
    X_test_tensor = X_test_tensor.to(device)
    y_test_tensor = y_test_tensor.to(device)
    outputs = model(X_test_tensor)
    predicted_labels = torch.round(torch.sigmoid(outputs))
    accuracy = (predicted_labels == y_test_tensor.unsqueeze(1)).sum().item() / len(y_test_tensor)
    print(f"Accuracy: {accuracy}")

    torch.save(model.state_dict(), output_name)

In [3]:
train_model("csvs/ml_X.csv", "built_models/torch_ml.pth")

Epoch 1/25, Loss: 0.7046518325805664
Epoch 2/25, Loss: 0.6625663638114929
Epoch 3/25, Loss: 0.7347057461738586
Epoch 4/25, Loss: 0.6465087532997131
Epoch 5/25, Loss: 0.6153339147567749
Epoch 6/25, Loss: 0.6178532242774963
Epoch 7/25, Loss: 0.5790522694587708
Epoch 8/25, Loss: 0.4673384130001068
Epoch 9/25, Loss: 0.5316528081893921
Epoch 10/25, Loss: 0.4821819067001343
Epoch 11/25, Loss: 0.37971585988998413
Epoch 12/25, Loss: 0.3720305263996124
Epoch 13/25, Loss: 0.36047109961509705
Epoch 14/25, Loss: 0.299319863319397
Epoch 15/25, Loss: 0.19170501828193665
Epoch 16/25, Loss: 0.1633976548910141
Epoch 17/25, Loss: 0.12776239216327667
Epoch 18/25, Loss: 0.16060037910938263
Epoch 19/25, Loss: 0.24129904806613922
Epoch 20/25, Loss: 0.31730055809020996
Epoch 21/25, Loss: 0.20292876660823822
Epoch 22/25, Loss: 0.2362421751022339
Epoch 23/25, Loss: 0.25781700015068054
Epoch 24/25, Loss: 0.20259520411491394
Epoch 25/25, Loss: 0.24711929261684418
Accuracy: 0.8539522662579374


In [4]:
train_model("csvs/ou_X.csv", "built_models/torch_ou.pth")

Epoch 1/25, Loss: 0.6846926212310791
Epoch 2/25, Loss: 0.686164379119873
Epoch 3/25, Loss: 0.6901816129684448
Epoch 4/25, Loss: 0.7051737308502197
Epoch 5/25, Loss: 0.6943862438201904
Epoch 6/25, Loss: 0.6810826659202576
Epoch 7/25, Loss: 0.6989530324935913
Epoch 8/25, Loss: 0.6823394298553467
Epoch 9/25, Loss: 0.6986621618270874
Epoch 10/25, Loss: 0.6987523436546326
Epoch 11/25, Loss: 0.7051970362663269
Epoch 12/25, Loss: 0.6704551577568054
Epoch 13/25, Loss: 0.7172917127609253
Epoch 14/25, Loss: 0.7054828405380249
Epoch 15/25, Loss: 0.6946144104003906
Epoch 16/25, Loss: 0.6747679710388184
Epoch 17/25, Loss: 0.7058021426200867
Epoch 18/25, Loss: 0.6831743717193604
Epoch 19/25, Loss: 0.687462329864502
Epoch 20/25, Loss: 0.6935343146324158
Epoch 21/25, Loss: 0.6808222532272339
Epoch 22/25, Loss: 0.6937941908836365
Epoch 23/25, Loss: 0.6975142359733582
Epoch 24/25, Loss: 0.6904025077819824
Epoch 25/25, Loss: 0.681022047996521
Accuracy: 0.5304424003504161


In [5]:
train_model("csvs/pl_X.csv", "built_models/torch_pl.pth")

Epoch 1/25, Loss: 0.691858172416687
Epoch 2/25, Loss: 0.6279863119125366
Epoch 3/25, Loss: 0.5955326557159424
Epoch 4/25, Loss: 0.6212185621261597
Epoch 5/25, Loss: 0.47834861278533936
Epoch 6/25, Loss: 0.7417680025100708
Epoch 7/25, Loss: 0.6848718523979187
Epoch 8/25, Loss: 0.47728240489959717
Epoch 9/25, Loss: 0.5076286196708679
Epoch 10/25, Loss: 0.6366924047470093
Epoch 11/25, Loss: 0.6650286912918091
Epoch 12/25, Loss: 0.5854007005691528
Epoch 13/25, Loss: 0.32653507590293884
Epoch 14/25, Loss: 0.34622010588645935
Epoch 15/25, Loss: 0.41398853063583374
Epoch 16/25, Loss: 0.41145074367523193
Epoch 17/25, Loss: 0.5393095016479492
Epoch 18/25, Loss: 0.6198967099189758
Epoch 19/25, Loss: 0.29599249362945557
Epoch 20/25, Loss: 0.4655645787715912
Epoch 21/25, Loss: 0.40670251846313477
Epoch 22/25, Loss: 0.2215307056903839
Epoch 23/25, Loss: 0.13756684958934784
Epoch 24/25, Loss: 0.24159133434295654
Epoch 25/25, Loss: 0.22614727914333344
Accuracy: 0.5450963222416813
